In [29]:
import numpy as np
from tkinter import *
import tkinter.scrolledtext as tkst
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.animation import FuncAnimation
import math
import pandas as pd

In [30]:
marathon_2015_2017_qualifying = pd.read_csv("./marathon_qualifying.csv")
marathon_2015_2016 \
= marathon_2015_2017_qualifying[marathon_2015_2017_qualifying['Year'] != 2017]
marathon_2017 \
= marathon_2015_2017_qualifying[marathon_2015_2017_qualifying['Year'] == 2017]

df_2015_2016 = pd.DataFrame(marathon_2015_2016, columns
                           =['M/F', 'Age', 'Pace', 'qualifying'])
df_2017 = pd.DataFrame(marathon_2017, columns
                           =['M/F', 'Age', 'Pace', 'qualifying'])
record_2015_2016 = df_2015_2016.values.tolist()
record_2017 = df_2017.values.tolist()
gender_list = ['Female', 'Male']

In [31]:
fig = Figure( figsize=(10,4), dpi=100)
ax = fig.add_subplot(111)
ax.set_xlim(15,88) 
ax.set_ylim(0,1300) 
ax.set_ylabel("Pace : Runner's overall minute per mile pace")
ax.set_xlabel("Age : Age on race day")

Text(0.5, 0, 'Age : Age on race day')

In [32]:
def normalization(record):
    r0 = record[0]
    r1 = record[1] /10
    r2 = record[2] /100
    return [r0, r1, r2]

x_train = [ normalization(r[0:3]) for r in record_2015_2016]
y_train = [ r[-1] for r in record_2015_2016]
x_test = [ r[0:3] for r in record_2017]
y_test = [ r[-1] for r in record_2017]
print(x_train[0], y_train[0])

[1, 2.5, 2.96] 1


In [33]:
def histogram():
    t_a = int(t_aSpbox.get()) -1 
    runner = x_test[t_a]
    t_g = int(runner[0])
    t_y = int(runner[1])
    t_p = int(runner[2])
    if(t_g): gender_color = 'b'
    else: gender_color = 'r'
    gender_record = df_2017[df_2017['M/F'] == t_g]
    gender_age_record = gender_record[gender_record.Age == t_y]
    gender_age_record_list = gender_age_record.values.tolist()
    ax.plot(gender_record.Age, gender_record.Pace, '.', color=gender_color, alpha=0.2)
    ax.plot(t_y, t_p, 'yd')
    stat = gender_age_record['Pace'].describe()
    title = 'Gender : '+gender_list[t_g]+', Age : '+str(t_y)+', Pace : '+str(t_p)
    ax.set_title(title)
    ax.annotate('['+gender_list[t_g]+', '+str(t_y)+']', (75, 1200), fontsize=10)
    ax.annotate("%10s %7i"%('Count : ', stat[0]), (75, 1150), fontsize=10)
    ax.annotate("%10s %7.3f"%('25% : ', stat[1]), (75, 1100), fontsize=10)
    ax.annotate("%10s %7.3f"%('75% : ', stat[3]), (75, 1050), fontsize=10)
    ax.annotate("%10s %7.3f"%('75% : ', stat[5]), (75, 1000), fontsize=10)
    fig.canvas.draw()

In [34]:
def learning():
    import tensorflow as tf
    import numpy as np
    t_a = int(t_aSpbox.get()) -1
    runner = x_test[t_a] 
    t_g = int(runner[0]) 
    t_y = int(runner[1]) 
    t_p = int(runner[2]) 
    t_t = int(t_tSpbox.get())
    t_r = float(t_rSpbox.get())
    
    model = tf.keras.models.Sequential()
    # 학습에 필요한 activation을 선형회귀 적용이 아닌 0, 1을 판단할 sigmoid 사용
    model.add(tf.keras.layers.Dense(1, input_shape=(3,), activation='sigmoid'))
    sgd = tf.keras.optimizers.SGD(lr=t_r)
    # 둘 중 하나를 선택하는 확률계산도구 binary_crossentropy 사용 + 정확도(예측확률)
    model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
    model.summary()
    history = model.fit(np.array(x_train), np.array(y_train), epochs=t_t)
    
    # 학습내용 출력# 학습내용 출력
    log_ScrolledText.insert(END, '\n\nCost Decent\n\n', 'HEADER')
    log_ScrolledText.insert(END, '%10s %20s %20s' % ('Step','Cost','Accuracy(%)')+\
                                                                            '\n\n')
    for step in range(t_t):
        if step % 100 == 0:
            cost_val = history.history['loss'][step]
            a_val = history.history['accuracy'][step]
            log_ScrolledText.insert(END, "%10i %20.5f %20.7f" % (step, cost_val,\
                                                                a_val*100)+'\n')
    # 예측
    winner = [ t_g, t_y, t_p ]
    result = model.predict(np.array([normalization(winner)])) # 스케일링 데이터 입력
    print(result)
    log_ScrolledText.insert(END, '\n\n')
    log_ScrolledText.insert(END, "%10s %20s" % ('Value        ', \
                                        'Qualifying Prediction\n\n'), 'HEADER')
    if(result[0]>0.5):
        log_ScrolledText.insert(END, '%10.7f %20s' % (result[0], 'Qualifier\n\n'), \
                                                                        'Qualifier')
    else :
        log_ScrolledText.insert(END, '%10.7f %20s' % (result[0], 'DisQualifier\n\n'),\
                                                               'DisQualifier')

In [ ]:
main = Tk()
main.title("Logistic Regression Binary Classification")
main.geometry()

#윈도우 제목 라벨
label = Label(main, text='Logistic Regression Binary Classification')
label.config(font=("Courier", 18))
label.grid(row=0, column=0, columnspan=6)
# 순위를 지정할 스핀박스 & 라벨
t_aVal = IntVar(value=1)
t_aSpbox = Spinbox(main, textvariable=t_aVal, from_=0, to=len(x_test), increment=1)
t_aSpbox.grid(row=1, column=1)
t_aLabel=Label(main, text='Rank of runner : ')
t_aLabel.grid(row=1, column=0)
# 학습 횟수를 지정할 스핀박스 & 라벨
t_tVal = IntVar(value=5000)
t_tSpbox = Spinbox(main, textvariable=t_tVal, from_=0, to=10000, increment=1000)
t_tSpbox.grid(row=1, column=3)
t_tLabel=Label(main, text='Number of train : ')
t_tLabel.grid(row=1, column=2)
# 러닝 레이트를 지정할 스핀박스 & 라벨
t_rVal = DoubleVar(value=0.01)
t_rSpbox = Spinbox(main, textvariable=t_rVal, from_=0, to=1, increment=0.01)
t_rSpbox.grid(row=1, column=5)
t_rLabel=Label(main, text='Learning rate : ')
t_rLabel.grid(row=1, column=4)

# 그래프 그리는 버튼과 예측버튼버튼
btn1 = Button(main, text="Histogram", height=2,command=lambda:histogram())
btn1.grid(row=2, column=0, columnspan=3, sticky=(W,E))
btn2 = Button(main, text="Prediction", height=2,command=lambda:learning())
btn2.grid(row=2, column=3, columnspan=3, sticky=(W,E))

# 스크롤텍스트와 스타일 설정
grad_canvas = FigureCanvasTkAgg(fig, main)
grad_canvas.get_tk_widget().grid(row=3, column=0, columnspan=6)

log_ScrolledText = tkst.ScrolledText(main,height=15)
log_ScrolledText.grid(row=4, column=0, columnspan=6, sticky=(N, S, W, E))
log_ScrolledText.configure(font='굴림')
log_ScrolledText.tag_config('Qualifier', foreground='blue', font=("굴림", 16))
log_ScrolledText.tag_config('DisQualifier', foreground='red', font=("굴림", 16))
log_ScrolledText.tag_config('HEADER', foreground='black', font=("굴림", 14))
log_ScrolledText.tag_config('TITLE', foreground='black', font=("굴림", 18))

main.mainloop()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1)                 4         
Total params: 4
Trainable params: 4
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5000
1664/1664 [==============================] - 1s 532us/step - loss: 0.7456 - accuracy: 0.6976
Epoch 2/5000
1664/1664 [==============================] - 1s 532us/step - loss: 0.5001 - accuracy: 0.7449
Epoch 3/5000
1664/1664 [==============================] - 1s 537us/step - loss: 0.4893 - accuracy: 0.7583
Epoch 4/5000
1664/1664 [==============================] - 1s 521us/step - loss: 0.4727 - accuracy: 0.7689
Epoch 5/5000
1664/1664 [==============================] - 1s 547us/step - loss: 0.4643 - accuracy: 0.7756
Epoch 6/5000
1664/1664 [==============================] - 1s 546us/step - loss: 0.4494 - accuracy: 0.7863
Epoch 7/5000
1664/16

1664/1664 [==============================] - 1s 494us/step - loss: 0.2618 - accuracy: 0.9233
Epoch 74/5000
1664/1664 [==============================] - 1s 494us/step - loss: 0.2598 - accuracy: 0.9244
Epoch 75/5000
1664/1664 [==============================] - 1s 490us/step - loss: 0.2607 - accuracy: 0.9238
Epoch 76/5000
1664/1664 [==============================] - 1s 504us/step - loss: 0.2611 - accuracy: 0.9227
Epoch 77/5000
1664/1664 [==============================] - 1s 495us/step - loss: 0.2604 - accuracy: 0.9241
Epoch 78/5000
1664/1664 [==============================] - 1s 496us/step - loss: 0.2598 - accuracy: 0.9227
Epoch 79/5000
1664/1664 [==============================] - 1s 494us/step - loss: 0.2558 - accuracy: 0.9249
Epoch 80/5000
1664/1664 [==============================] - 1s 502us/step - loss: 0.2556 - accuracy: 0.9250
Epoch 81/5000
1664/1664 [==============================] - 1s 494us/step - loss: 0.2558 - accuracy: 0.9260
Epoch 82/5000
1664/1664 [==========================

1664/1664 [==============================] - 1s 569us/step - loss: 0.2242 - accuracy: 0.9286
Epoch 150/5000
1664/1664 [==============================] - 1s 525us/step - loss: 0.2224 - accuracy: 0.9289
Epoch 151/5000
1664/1664 [==============================] - 1s 502us/step - loss: 0.2226 - accuracy: 0.9279
Epoch 152/5000
1664/1664 [==============================] - 1s 507us/step - loss: 0.2217 - accuracy: 0.9286
Epoch 153/5000
1664/1664 [==============================] - 1s 523us/step - loss: 0.2212 - accuracy: 0.9291
Epoch 154/5000
1664/1664 [==============================] - 1s 521us/step - loss: 0.2231 - accuracy: 0.9274
Epoch 155/5000
1664/1664 [==============================] - 1s 511us/step - loss: 0.2220 - accuracy: 0.9266
Epoch 156/5000
1664/1664 [==============================] - 1s 500us/step - loss: 0.2204 - accuracy: 0.9284
Epoch 157/5000
1664/1664 [==============================] - 1s 494us/step - loss: 0.2213 - accuracy: 0.9274
Epoch 158/5000
1664/1664 [=================

Epoch 225/5000
1664/1664 [==============================] - 1s 502us/step - loss: 0.2061 - accuracy: 0.9284
Epoch 226/5000
1664/1664 [==============================] - 1s 503us/step - loss: 0.2101 - accuracy: 0.9276
Epoch 227/5000
1664/1664 [==============================] - 1s 532us/step - loss: 0.2092 - accuracy: 0.9274
Epoch 228/5000
1664/1664 [==============================] - 1s 513us/step - loss: 0.2106 - accuracy: 0.9267
Epoch 229/5000
1664/1664 [==============================] - 1s 508us/step - loss: 0.2089 - accuracy: 0.9272
Epoch 230/5000
1664/1664 [==============================] - 1s 501us/step - loss: 0.2085 - accuracy: 0.9265
Epoch 231/5000
1664/1664 [==============================] - 1s 506us/step - loss: 0.2055 - accuracy: 0.9293
Epoch 232/5000
1664/1664 [==============================] - 1s 509us/step - loss: 0.2081 - accuracy: 0.9276
Epoch 233/5000
1664/1664 [==============================] - 1s 506us/step - loss: 0.2072 - accuracy: 0.9277
Epoch 234/5000
1664/1664 [==

1664/1664 [==============================] - 1s 497us/step - loss: 0.2003 - accuracy: 0.9288
Epoch 301/5000
1664/1664 [==============================] - 1s 499us/step - loss: 0.2006 - accuracy: 0.9280
Epoch 302/5000
1664/1664 [==============================] - 1s 499us/step - loss: 0.1996 - accuracy: 0.9291
Epoch 303/5000
1664/1664 [==============================] - 1s 501us/step - loss: 0.1995 - accuracy: 0.9294
Epoch 304/5000
1664/1664 [==============================] - 1s 499us/step - loss: 0.1977 - accuracy: 0.9279
Epoch 305/5000
1664/1664 [==============================] - 1s 493us/step - loss: 0.1994 - accuracy: 0.9273
Epoch 306/5000
1664/1664 [==============================] - 1s 498us/step - loss: 0.2003 - accuracy: 0.9278
Epoch 307/5000
1664/1664 [==============================] - 1s 494us/step - loss: 0.2000 - accuracy: 0.9268
Epoch 308/5000
1664/1664 [==============================] - 1s 495us/step - loss: 0.1991 - accuracy: 0.9286
Epoch 309/5000
1664/1664 [=================

...

1664/1664 [==============================] - 1s 500us/step - loss: 0.1799 - accuracy: 0.9310
Epoch 754/5000
1664/1664 [==============================] - 1s 494us/step - loss: 0.1832 - accuracy: 0.9283
Epoch 755/5000
1664/1664 [==============================] - 1s 499us/step - loss: 0.1818 - accuracy: 0.9296
Epoch 756/5000
1664/1664 [==============================] - 1s 500us/step - loss: 0.1833 - accuracy: 0.9266
Epoch 757/5000
1664/1664 [==============================] - 1s 500us/step - loss: 0.1832 - accuracy: 0.9280
Epoch 758/5000
1664/1664 [==============================] - 1s 488us/step - loss: 0.1858 - accuracy: 0.9268
Epoch 759/5000
1664/1664 [==============================] - 1s 508us/step - loss: 0.1843 - accuracy: 0.9281
Epoch 760/5000
1664/1664 [==============================] - 1s 492us/step - loss: 0.1813 - accuracy: 0.9292
Epoch 761/5000
1664/1664 [==============================] - 1s 507us/step - loss: 0.1796 - accuracy: 0.9299
Epoch 762/5000
1664/1664 [=================

Epoch 829/5000
1664/1664 [==============================] - 1s 507us/step - loss: 0.1839 - accuracy: 0.9265
Epoch 830/5000
1664/1664 [==============================] - 1s 505us/step - loss: 0.1817 - accuracy: 0.9287
Epoch 831/5000
1664/1664 [==============================] - 1s 497us/step - loss: 0.1800 - accuracy: 0.9283
Epoch 832/5000
1664/1664 [==============================] - 1s 497us/step - loss: 0.1817 - accuracy: 0.9288
Epoch 833/5000
1664/1664 [==============================] - 1s 499us/step - loss: 0.1819 - accuracy: 0.9288
Epoch 834/5000
1664/1664 [==============================] - 1s 501us/step - loss: 0.1817 - accuracy: 0.9277
Epoch 835/5000
1664/1664 [==============================] - 1s 499us/step - loss: 0.1833 - accuracy: 0.9282
Epoch 836/5000
1664/1664 [==============================] - 1s 503us/step - loss: 0.1835 - accuracy: 0.9262
Epoch 837/5000
1664/1664 [==============================] - 1s 549us/step - loss: 0.1778 - accuracy: 0.93070s - l
Epoch 838/5000
1664/16

1664/1664 [==============================] - 1s 510us/step - loss: 0.1824 - accuracy: 0.9267
Epoch 905/5000
1664/1664 [==============================] - 1s 500us/step - loss: 0.1830 - accuracy: 0.9269
Epoch 906/5000
1664/1664 [==============================] - 1s 495us/step - loss: 0.1807 - accuracy: 0.9285
Epoch 907/5000
1664/1664 [==============================] - 1s 506us/step - loss: 0.1816 - accuracy: 0.9273
Epoch 908/5000
1664/1664 [==============================] - 1s 497us/step - loss: 0.1839 - accuracy: 0.9274
Epoch 909/5000
1664/1664 [==============================] - 1s 500us/step - loss: 0.1824 - accuracy: 0.9274
Epoch 910/5000
1664/1664 [==============================] - 1s 489us/step - loss: 0.1784 - accuracy: 0.9300
Epoch 911/5000
1664/1664 [==============================] - 1s 504us/step - loss: 0.1812 - accuracy: 0.9290
Epoch 912/5000
1664/1664 [==============================] - 1s 511us/step - loss: 0.1808 - accuracy: 0.9287
Epoch 913/5000
1664/1664 [=================

Epoch 980/5000
1664/1664 [==============================] - 1s 503us/step - loss: 0.1812 - accuracy: 0.9270
Epoch 981/5000
1664/1664 [==============================] - 1s 498us/step - loss: 0.1813 - accuracy: 0.9275
Epoch 982/5000
1664/1664 [==============================] - 1s 508us/step - loss: 0.1805 - accuracy: 0.9264
Epoch 983/5000
1664/1664 [==============================] - 1s 500us/step - loss: 0.1806 - accuracy: 0.9291
Epoch 984/5000
1664/1664 [==============================] - 1s 501us/step - loss: 0.1829 - accuracy: 0.9279
Epoch 985/5000
1664/1664 [==============================] - 1s 507us/step - loss: 0.1827 - accuracy: 0.9272
Epoch 986/5000
1664/1664 [==============================] - 1s 503us/step - loss: 0.1806 - accuracy: 0.9264
Epoch 987/5000
1664/1664 [==============================] - 1s 499us/step - loss: 0.1802 - accuracy: 0.9275
Epoch 988/5000
1664/1664 [==============================] - 1s 495us/step - loss: 0.1786 - accuracy: 0.9289
Epoch 989/5000
1664/1664 [==

1664/1664 [==============================] - 1s 508us/step - loss: 0.1842 - accuracy: 0.9257
Epoch 1056/5000
1664/1664 [==============================] - 1s 539us/step - loss: 0.1829 - accuracy: 0.9255
Epoch 1057/5000
1664/1664 [==============================] - 1s 498us/step - loss: 0.1805 - accuracy: 0.9292
Epoch 1058/5000
1664/1664 [==============================] - 1s 502us/step - loss: 0.1789 - accuracy: 0.9285
Epoch 1059/5000
1664/1664 [==============================] - 1s 497us/step - loss: 0.1823 - accuracy: 0.9272
Epoch 1060/5000
1664/1664 [==============================] - 1s 499us/step - loss: 0.1812 - accuracy: 0.9271
Epoch 1061/5000
1664/1664 [==============================] - 1s 494us/step - loss: 0.1791 - accuracy: 0.9298
Epoch 1062/5000
1664/1664 [==============================] - 1s 501us/step - loss: 0.1804 - accuracy: 0.9274
Epoch 1063/5000
1664/1664 [==============================] - 1s 507us/step - loss: 0.1772 - accuracy: 0.9280
Epoch 1064/5000
1664/1664 [========